Импорт библиотек и установка устройства

In [1]:
# Импорт необходимых библиотек
import ffmpeg
import os
import torch
import whisper
import cv2
from PIL import Image
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, 
    MarianMTModel, MarianTokenizer, 
    BartForConditionalGeneration, BartTokenizer, 
    BlipProcessor, BlipForConditionalGeneration
)
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Выбор устройства для вычислений (GPU или CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"


Загрузка моделей и токенизаторов

In [2]:
# Загрузка моделей и токенизаторов
whisper_model = whisper.load_model("small", device=device)  # Модель распознавания речи

translation_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ru-en").to(device)
translation_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")  # Модель и токенизатор для перевода с русского на английский

back_translation_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ru").to(device)
back_translation_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")  # Модель и токенизатор для перевода с английского на русский

summarization_model_name = 'facebook/bart-large-cnn'
summarization_model = BartForConditionalGeneration.from_pretrained(summarization_model_name).to(device)
summarization_tokenizer = BartTokenizer.from_pretrained(summarization_model_name)  # Модель и токенизатор для суммаризации текста


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Функция для нормализации текста

In [3]:
def normalize_text(text):
    """Нормализация текста: удаление пробелов и нежелательных символов"""
    text = text.strip().lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text


Обработка аудио

In [4]:
def process_audio(video_path):
    """Обработка аудио для RuBertAudio: транскрибация, перевод и суммаризация"""
    temp_audio_path = "temp_audio.wav"
    
    try:
        # Извлечение аудио из видео
        ffmpeg.input(video_path, t=90).output(temp_audio_path, acodec='pcm_s16le', ac=1, ar='16000').run(quiet=True)
    except ffmpeg.Error as e:
        print(e.stderr.decode())
        raise

    # Транскрибация аудио
    transcript = whisper_model.transcribe(temp_audio_path)
    os.remove(temp_audio_path)

    # Перевод на английский
    english_text = translator_to_english(transcript['text'])
    # Суммаризация
    summary_english = summarize(english_text)
    # Перевод обратно на русский
    summarized_text = translate_to_russian(summary_english)

    return summarized_text


Функции перевода

In [5]:
def translator_to_english(text):
    """Перевод текста с русского на английский"""
    inputs = translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated = translation_model.generate(**inputs)
    return translation_tokenizer.decode(translated[0], skip_special_tokens=True)

def translate_to_russian(text):
    """Перевод текста с английского на русский"""
    inputs = back_translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated_back = back_translation_model.generate(**inputs)
    return back_translation_tokenizer.decode(translated_back[0], skip_special_tokens=True)


Суммаризация текста

In [6]:
def summarize(text):
    """Суммаризация текста"""
    inputs = summarization_tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = summarization_model.generate(inputs['input_ids'], max_length=300, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


Обработка видео кадров

In [7]:
def process_video_frames(video_path):
    """Обработка видео кадров и получение текстовых описаний"""
    # Загрузка моделей для генерации описаний
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    caption_model.to(device)

    # Извлечение и обработка кадров видео
    frames = extract_frames(video_path)
    captions = generate_captions(frames, processor, caption_model, device)
    combined_text = ", ".join(captions)

    # Суммаризация и перевод описаний
    summarized_text = summarize(combined_text)
    translated_description = translate_to_russian(summarized_text)

    return translated_description


Извлечение и обработка кадров

In [8]:
def extract_frames(video_path, num_frames=16):
    """Извлечение кадров из видео"""
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(total_frames // num_frames, 1)
    success, image = vidcap.read()
    count = 0
    while success and len(frames) < num_frames:
        if count % frame_interval == 0:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(image_rgb))
        success, image = vidcap.read()
        count += 1
    vidcap.release()
    return frames

def generate_captions(frames, processor, model, device):
    """Генерация описаний для каждого из кадров"""
    captions = []
    unique_captions = set()
    inputs = processor(images=frames, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=30, num_beams=5, early_stopping=True)
    for output in outputs:
        caption = processor.decode(output, skip_special_tokens=True)
        if caption not in unique_captions:
            unique_captions.add(caption)
            captions.append(caption)
    return captions


Предсказание тегов

In [9]:
def predict_tags(model_path, input_text):
    """Предсказание тегов с использованием обученной модели"""
    # Загрузка модели и токенизатора
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.to(device)

    # Нормализация и токенизация текста
    normalized_text = normalize_text(input_text)
    encoding = tokenizer(normalized_text, return_tensors='pt', truncation=True, padding=True, max_length=256)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    # Предсказание
    with torch.no_grad():
        outputs = model(**encoding)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Загрузка кодировщика меток
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load(os.path.join(model_path, 'label_encoder_classes.npy'), allow_pickle=True)

    # Получение топовых тегов
    top_k = 5
    probabilities, indices = torch.topk(probabilities, top_k, dim=1)
    tags_with_probs = [(label_encoder.inverse_transform([idx.item()])[0], prob.item()) for prob, idx in zip(probabilities[0], indices[0])]

    return tags_with_probs


In [23]:
from collections import defaultdict

def combine_and_select_top_tags(text_tags, audio_tags, video_tags, top_n=5):
    """Объединение и выбор самых уверенных тегов от трех различных моделей, без повторений"""
    
    # Объединение всех тегов в один список
    all_tags = text_tags + audio_tags + video_tags
    
    # Используем словарь для хранения максимальной уверенности для каждого тега
    tag_confidence = defaultdict(lambda: 0)

    # Обновляем словарь, сохраняя максимальную уверенность для каждого тега
    for tag, accuracy in all_tags:
        if accuracy > tag_confidence[tag]:
            tag_confidence[tag] = accuracy

    # Преобразуем в список и сортируем по убыванию уверенности
    sorted_tags = sorted(tag_confidence.items(), key=lambda item: item[1], reverse=True)
    
    # Выбираем топ-N наиболее уверенных тегов
    most_confident_tags = sorted_tags[:top_n]

    # Выводим наиболее уверенные теги и их точность
    print("Наиболее уверенные теги и их точность:")
    for tag, accuracy in most_confident_tags:
        print(f"{tag}: {accuracy:.2f}")

    return most_confident_tags


Основная логика выполнения

In [24]:
def main(video_path, title, description):
    """Основная функция для обработки видео и получения тегов"""
    
    # Обработка текста
    print("Processing text...")
    combined_text = f"{title} {description}"
    text_tags = predict_tags("RuBertText/saved_model", combined_text)
    print("Text model tags:", text_tags)

    # Обработка аудио
    print("Processing audio...")
    summarized_audio_text = process_audio(video_path)
    audio_tags = predict_tags("RuBertAudio/saved_model", summarized_audio_text)
    print("Audio model tags:", audio_tags)

    # Обработка видео
    print("Processing video frames...")
    translated_video_description = process_video_frames(video_path)
    video_tags = predict_tags("RuBertVideo/saved_model", translated_video_description)
    print("Video model tags:", video_tags)

    # Объединяем результаты всех моделей
    combine_and_select_top_tags(text_tags, audio_tags, video_tags)


Запуск основного метода

In [28]:
# Пример использования
video_path = "drift.mp4"
title = "Название"
description = "Описание"
main(video_path, title, description)


Processing text...
Text model tags: [(np.str_('Массовая культура: Юмор и сатира'), 0.5076751708984375), (np.str_('Массовая культура'), 0.2455354630947113), (np.str_('Еда и напитки: Кулинария'), 0.01389509066939354), (np.str_('Спорт: Рыбалка'), 0.008057245053350925), (np.str_('Путешествия'), 0.007944507524371147)]
Processing audio...
Audio model tags: [('Массовая культура: Юмор и сатира', 0.9809638261795044), ('Массовая культура', 0.0017407310660928488), ('Еда и напитки: Кулинария', 0.0014596988912671804), ('Образование: Онлайн-образование', 0.0010987541172653437), ('Религия и духовность: астрология', 0.0010873978026211262)]
Processing video frames...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Video model tags: [('Массовая культура', 0.6661799550056458), ('Массовая культура: Юмор и сатира', 0.12876959145069122), ('Транспорт', 0.02259010262787342), ('Путешествия', 0.014793174341320992), ('События и достопримечательности: Исторические места и достопримечательности', 0.007212325930595398)]
Наиболее уверенные теги и их точность:
Массовая культура: Юмор и сатира: 0.98
Массовая культура: 0.67
Транспорт: 0.02
Путешествия: 0.01
Еда и напитки: Кулинария: 0.01
